In [6]:
from random import randint

for i in range(1,11):
    print(randint(1,10))

1
5
2
3
5
3
9
2
8
8


In [11]:
pc_ref = [0, 25, -60, 400, -20, -75, 0, 300, -62.5, 66.7, 0, 40, -42.9, -25, -66.7, 0, 0, 700, -12.5, -28.6]
pc_tst = [0, 400, -60, 400, -20, -75, 0, 0, 300, 0]

def get_chunks(lst):
    r=[]
    chunk_size = 3
    nxt_chunk_end_idx = chunk_size - 1
    tot = len(lst)
    for idx, val in enumerate(lst):
        end_idx = idx+chunk_size
        if end_idx <= tot:
            r.append(lst[idx:end_idx])
    return r

ref_chunks = get_chunks(pc_ref)
tst_chunks = get_chunks(pc_tst)

In [12]:
print(ref_chunks)
print(tst_chunks)

[[0, 25, -60], [25, -60, 400], [-60, 400, -20], [400, -20, -75], [-20, -75, 0], [-75, 0, 300], [0, 300, -62.5], [300, -62.5, 66.7], [-62.5, 66.7, 0], [66.7, 0, 40], [0, 40, -42.9], [40, -42.9, -25], [-42.9, -25, -66.7], [-25, -66.7, 0], [-66.7, 0, 0], [0, 0, 700], [0, 700, -12.5], [700, -12.5, -28.6]]
[[0, 400, -60], [400, -60, 400], [-60, 400, -20], [400, -20, -75], [-20, -75, 0], [-75, 0, 0], [0, 0, 300], [0, 300, 0]]
